<a href="https://colab.research.google.com/github/Baldezo313/Some-real-world-machine-learning-project/blob/main/PROJET_14_AUTOMATISATION_DU_WORKFLOW_D'UN_PROJET_DE_MACHINE_LEARNING_AVEC_LA_FONCTION_PIPELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJET 14 : AUTOMATISATION DU WORKFLOW D'UN PROJET DE MACHINE LEARNING AVEC LA FONCTION PIPELINE  

Un projet en Data Science nécessite plusieurs étapes pour parvenir aux résultats finaux. Trois étapes sont incontournables : Nettoyage des données, Transformation (Prétraitement) des
données et Modélisation. La réalisation de ces étapes n'est pas toujours linéaire. Bien souvent, un projet en Data Science est un processus itératif c'est-à-dire qu'il faut effectuer des feedbacks réguliers afin de revoir tel ou tel autre élément pour augmenter la performance de votre modèle.  
De ce fait, le travail peut vite devenir fastidieux, très consommateur de temps et pénible. De plus, lorsque vous avez effectué séparément chacune de ces étapes pour une dataframe donnée, il faudra encore les reprendre pour un autre jeu de données (Travail répétitif).  

Le module `pipeline de sklearn` avec sa fonction `Pipeline` permet de construire un super objet composé de transformateurs de la dataframe et d'un ou plusieurs estimateurs. En clair, vous
pouvez avoir par exemple les étapes de prétraitement, de transformation (Réduction de dimensionnalité par exemple) et de modélisation le tout dans un même objet appelé pipeline.
Ainsi, cet objet (pipeline) peut être appliqué facilement à d'autres jeux de données. Alors, grâce à l'objet pipeline, vous gagnez beaucoup de temps et vous éliminez d'un coup la pénibilité et la répétitivité du travail.    


Après la lecture de ce projet, vous saurez comment automatiser toutes vos tâches de Data Science et de Machine Learning. De plus, la fonction pipeline n'aura plus aucun secret pour vous

## ETUDE DE CAS D'APPLICATION DE PIPELINE : MODELISATION DU RISQUE DE CREDIT  

Vous êtes Data Scientist dans une grande Banque et vous êtes chargé principalement de la modélisation du risque de crédit. Vous devez effectuer plusieurs tâches pour augmenter la
performance de vos modèles. Pour chaque nouveau jeu de données, vous devez encore reprendre toutes ces tâches. Cela vous prend énormément de temps et la pression devient de plus en forte par rapport aux deadlines.  

Plus d'inquiétude à vous faire. Après la lecture de ce projet, vous saurez comment automatiser toutes vos tâches de Data Science et de Machine Learning. De plus, la fonction pipeline n'aura plus aucun secret pour vous et vous serez toujours en avance sur les délais  

Commençons par importer les librairies nécessaires pour cette étude.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

**DONNEES**  

Avant d'importer le jeu de données dans votre notebook, il est conseillé de l'ouvrir d'abord avec Excel ou toute autre application afin de faire une petite inspection visuelle. De plus, il faut lire la documentation sur vos données si celle-ci existe. C'est ce travail préalable qui nous a permis de savoir qu'il n'y a pas de noms de colonnes dans le jeu de données, que le séparateur est une virgule et que les valeurs manquantes sont
représentées par un "?".

In [2]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/Baldezo313/Some-real-world-machine-learning-project/refs/heads/main/Machine-Learning-par-la-pratique-avec-Python-master/crx.data?token=GHSAT0AAAAAAC4VKTEQW4DYFINZZ7HSCSNWZ4OKNWA',
sep=',', header = None, na_values = "?")
df_raw.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [3]:
#Informations sur le jeu de données
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    float64
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      677 non-null    float64
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 86.4+ KB


### PREPARATION DES DONNEES  

La préparation des données est une étape très importante dans tout projet en Data Science.  
Concernant les valeurs manquantes, elles seront tout simplement retirées. Mais il est bon de savoir qu'il existe plusieurs manières de traiter les données manquantes comme les imputations
par la moyenne, médiane, valeur la plus fréquente, etc.  

Créons une copie de notre jeu de données :

In [4]:
df = df_raw.copy()

Selon la description de ce jeu de données, au niveau de la colonne 15 le symbole "+" représente un cas où le crédit a été accordé (client solvable) et le symbole "-" représente un cas où le crédit n'a pas été accordé (client insolvable) à une personne. Nous allons remplacer ces symboles : 1 pour "+" et 0 pour "-".

In [5]:
df[15].replace({"+":1, "-":0}, inplace=True)

<ipython-input-5-8678dc839ed0>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[15].replace({"+":1, "-":0}, inplace=True)
<ipython-input-5-8678dc839ed0>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[15].replace({"+":1, "-":0}, inplace=True)


* Nombre de valeurs manquantes par colonne :

In [7]:
df.isna().sum().sort_values(ascending=False)

,0
13,13
0,12
1,12
5,9
6,9
3,6
4,6
2,0
7,0
8,0


* Retirons toutes les lignes présentant des données manquantes :

In [8]:
df.dropna(axis=0, inplace=True)
df.shape

(653, 16)

En éliminant les données manquantes, on passe de 690 à 653 lignes.

### ETAPE DE PRETRAITEMENT DES DONNEES  

Le prétraitement des données (**Data Preprocessing**) est l'étape qui vient juste avant la modélisation proprement dite. Dans le présent cas, cette étape comprendra la transformation
des variables catégorielles en variables numériques (**Encoding**) ainsi que la standardisation (**Scaling**).  

L'objectif étant de construire un pipeline qui englobe tout le processus de Machine Learning, le prétraitement sera donc la première composante (ou étape) de notre pipeline.  

Pour la transformation des variables catégorielles, la fonction  [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) a été utilisé

In [9]:
#Création d'un objet OneHotEncoder pour transformer les variables catégorielles
catTransformer = OneHotEncoder(drop='first')



L'argument *drop = 'first'* permet d'éliminer la variable générée à partir de la première modalité.  
Cet argument est important car il permet de supprimer le problème de colinéarité entre variables générées après un *One Hot Enconding*.  

Pour la standardisation au niveau des variables numériques, la fonction [StandardScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) a
été utilisé.

In [10]:
#Création d'un objet StandardScaler pour mettre à la même échelle les variables numériques
numTransformer = StandardScaler()

Appliquons maintenant ces transformateurs. Au lieu d'appliquer séparément chaque transformateur, nous allons utiliser la fonction [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) qui nous permettra d'appliquer à la fois les deux transformateurs en les regroupant dans un même objet.

In [11]:
#Features
X = df.loc[:, 0:14]

#Target
y = df.loc[:, 15]

#Variables catégorielles
catVariables = X.select_dtypes(include=['object']).columns


#Variabes numériques
numVariables = X.select_dtypes(exclude=['object']).columns


#Création d'un objet ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numTransformer, numVariables),
         ('categoric', catTransformer, catVariables)
         ]
    )

* Créons l'objet pipeline avec comme étape l'objet preprocessor :

In [12]:
#Création d'un objet pipeline qui inclut l'objet ColumnTrasformer
pipeline_object = Pipeline(steps=[('preprocessing', preprocessor)])

### ETAPE DE REDUCTION DE DIMENSIONNALITE  

Mettons à jour notre objet pipeline en y incluant un objet PCA *(Principal Components Analysis)* comme deuxième étape :

In [13]:
pipeline_object = Pipeline(
    steps=[
        ('preprocessing', preprocessor),
         ('dim_red', PCA(n_components=10))
         ]
    )

L'étape de modélisation permettra de conférer à cet objet pipeline le caractère d'un estimateur.

### ETAPE DE CONSTRUCTION DU MODELE  

Avant de passer à la modélisation, divisons la dataframe en données d'entraînement et d'évaluation de modèle.  



In [14]:
#Train/Test set
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = seed, test_size = 0.2)

* **UN SEUL ALGORITHME DE CLASSIFICATION**  

Mettons à jour notre objet pipeline en y incluant un modèle de classification utilisant l'algorithme de régression logistique.

In [15]:
pipeline_object = Pipeline(
    steps=[
        ('preprocessing', preprocessor),
         ('dim_red', PCA(n_components=10)),
          ('lr_model', LogisticRegression(random_state = seed))
          ]
    )

Le fait d'avoir ajouté un algorithme de classification en dernière étape à l'objet pipeline lui confère les caractéristiques d'un classificateur (ici Régression Logistique)  

Maintenant nous pouvons utiliser l'objet pipeline comme estimateur et lui appliquer les méthodes ***fit()*** et ***predict()*** et terminer par le calcul de certaines métriques pour évaluer le modèle.

In [16]:
#Application du pipeline aux données d'entraînement
pipeline_object.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  Index([1, 2, 7, 10, 13, 14], dtype='int64')),
                                                 ('categoric',
                                                  OneHotEncoder(drop='first'),
                                                  Index([0, 3, 4, 5, 6, 8, 9, 11, 12], dtype='int64'))])),
                ('dim_red', PCA(n_components=10)),
                ('lr_model', LogisticRegression(random_state=42))])

Dans un premier temps, les données d'entraînement subiront le prétraitement (première étape du Pipeline). Dans un second temps elles subiront une réduction de dimensions grâce à l'objet
PCA et en troisième temps elles seront passées dans l'algorithme de Régression logistique.   

Pour évaluer le modèle, on peut tout simplement utiliser la méthode score() comme suit :

In [17]:
#Score du modèle
print("Accuracy on Training Data:", round(pipeline_object.score(X_train, y_train), 2))

Accuracy on Training Data: 0.86


In [18]:
print("Accuracy on Test Data:", round(pipeline_object.score(X_test, y_test), 2))

Accuracy on Test Data: 0.8


Le modèle donne un score de 86% sur le train data contre 80% pour le test data.  

Passons aux prédictions du modèle :

In [19]:
# Prédictions
yhat = pipeline_object.predict(X_test)

On peut dès lors dresser un rapport complet d'évaluation du modèle :

In [20]:
#Rapport de classification
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.84      0.82      0.83        76
           1       0.75      0.78      0.77        55

    accuracy                           0.80       131
   macro avg       0.80      0.80      0.80       131
weighted avg       0.80      0.80      0.80       131



Selon ce rapport de performance du modèle, 80% des clients du test data ont été correctement classé comme solvables ou non. 83% des clients insolvables (classe 0) ont été correctement
classés comme tel ce qui veut dire que 17% des clients insolvables ont été mal classés comme étant des clients solvables : c'est un risque de perte d'argent de l'ordre de 17% pour la banque.  

Par ailleurs 78% des clients solvables (classe 1) ont été classé comme tel ce qui veut dire qu'environ 22% des clients solvables ont été mal classés comme étant des clients insolvables :
c'est une perte d'opportunité pour la banque de l'ordre de 22%.  




* **PLUSIEURS ALGORITHMES DE CLASSIFICATION**  

Dans un projet de Data Science, il est très rare d'essayer un seul modèle pour tirer des conclusions. Plusieurs algorithmes sont entraînés par les données. Le modèle le plus performant
est sélectionné sur la base du calcul d'un paramètre d'évaluation préalablement défini selon le type de Business.  

Grâce à une boucle *for*, nous allons entraîner plusieurs modèles et évaluer leur performance.  
Voici le code que vous pouvez écrire pour ce genre de tâche :

In [21]:
#Création d'une liste d'algorithmes de classification
classifiers = [
    LogisticRegression(random_state=seed),
    RandomForestClassifier(random_state=seed),
    AdaBoostClassifier(random_state=seed),
    KNeighborsClassifier(n_neighbors=5)
    ]


#Entraînement et évaluation de chacun des algorithmes de la liste ci-dessus
for algorithm in classifiers:
  pipeline_object = Pipeline(
      steps=[('preprocessing', preprocessor),
       ('dimred', PCA(n_components=10)),
        ('model', algorithm)])

  #Fit
  pipeline_object.fit(X_train, y_train)


  #Score
  print(algorithm)
  print("Accuracy on Training Data:", round(pipeline_object.score(X_train, y_train), 2))
  print("Accuracy on Test Data:", round(pipeline_object.score(X_test, y_test), 2))
  print("-" * 90)



LogisticRegression(random_state=42)
Accuracy on Training Data: 0.86
Accuracy on Test Data: 0.8
------------------------------------------------------------------------------------------
RandomForestClassifier(random_state=42)
Accuracy on Training Data: 1.0
Accuracy on Test Data: 0.82
------------------------------------------------------------------------------------------
AdaBoostClassifier(random_state=42)
Accuracy on Training Data: 0.88
Accuracy on Test Data: 0.83
------------------------------------------------------------------------------------------
KNeighborsClassifier()
Accuracy on Training Data: 0.85
Accuracy on Test Data: 0.8
------------------------------------------------------------------------------------------


Vous pouvez essayer plusieurs autres algorithmes avec leurs hyperparamètres par défaut. L'idée est de pouvoir sélectionner les algorithmes qui sont performants aussi bien sur le train data que sur le test data. La seule contrainte possible est le temps d'exécution qui est directement rattachée à la puissance de l'ordinateur que vous utilisez. Si vous utilisez une machine de guerre :), par exemple un core i9 avec 64 GB de ram , faites libre cours à votre imagination. De toute façon, il y a une multitude d'algorithmes de Machine Learning.  

En une seule fois, nous avons entraîné 04 algorithmes de classification ce qui fait un gain de temps énorme (On pouvait même inclure d'autres algorithmes). Nous retrouvons les mêmes
scores pour la régression logistique.  

Au vu des scores des différents modèles sur le training et le test set, le modèle de **AdaBoostClassifier** semble être le plus performant.

Les algorithmes ci-dessus ont été entraîné avec leurs paramètres par défaut. L'une des étapes clé d'un projet de Data Science consiste à affiner un modèle en essayant plusieurs
hyperparamètres (**Hyperparameters Tuning**). Cette tâche est non seulement fastidieuse mais très consommatrice en temps. De plus, il faut aussi effectuer une  [cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html) afin de se
prémunir contre le surapprentissage ( [Overfitting](https://en.wikipedia.org/wiki/Overfitting)).  

Heureusement, cette fois ci-encore nous avons pipeline . Nous allons implémenter dans un pipeline toutes ces tâches de recherche des meilleurs hyperparamètres et de cross-validation au
niveau de l'algorithme de AdaBoostClassifier afin d'essayer d'augmenter sa performance.

In [ ]:
#Création d'un pipeline avec RandomForestClassifier
pipe = Pipeline(
    steps=[
        ('preprocessing', preprocessor),
         ('dimred', PCA()),
          ('classifier', LogisticRegression(random_state=seed))
          ]
    )
#Création d'un dictionnaire regroupant les paramètres à tourner

param_grid ={
    'dimred__n_components' : range(2, 16),
    'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : [1,3, 5],
    'classifier__solver' : ['liblinear'],
    'classifier__max_iter' : [100, 500]
    }

#Création d'un estimateur avec GridSarchCV (Combinaison de Grid Search et de Cross-Validation)
estimator = GridSearchCV(pipe, cv=10, param_grid=param_grid)

#Entraînement de cet estimateur
estimator.fit(X_train,y_train)


#Meilleur score et meilleurs paramètres
print("Meilleur score obtenu: %f avec les hyperparamètres : %s" % (estimator.best_score_, estimator.best_params_))

Notez qu'avec le pipeline, on a pu défini un espace de recherche aussi bien pour les hyperparamètres de l'algorithme mais aussi une gamme de nombre de composantes principales.  

Puisque l'argument **refit** de la fonction **GridSearchCV** est par défaut égal à **True**, alors le modèle est automatiquement réentraîné avec les meilleurs hyperparamètres trouvés. On peut
donc effectuer directement des prédictions et aussi dresser le rapport de classification :

In [ ]:
#Prédiction sur le test set avec le meilleur estimateur
pred = estimator.predict(X_test)


#Rapport de classification
print(classification_report(pred, y_test))

Selon ce rapport de performance du modèle, 84% (contre un score de 85% avec les paramètres par défaut) des clients du test data ont été correctement classé comme solvables ou non . Avec ce modèle tourné, on prédit mieux les clients insolvables. En effet 90% des clients insolvables (classe 0) ont été correctement classés comme tel ce qui veut dire que 10% des clients
 insolvables ont été mal classés comme étant des clients solvables : c'est un risque de perte d'argent de l'ordre de 10% (contre 12% avec le modèle par défaut) pour la banque. En revanche, on perd en justesse sur la prédiction des clients solvables. 76% (contre 81% avec le modèle par défaut) des clients solvables (classe 1) ont été classé comme tel ce qui veut dire qu'environ 24% des clients solvables ont été mal classés comme étant des clients insolvables : c'est une perte d'opportunité pour la banque de l'ordre de 24%.  

 Avec le premier modèle de régression logistique (hyperparamètres par défaut), la Banque a un risque de perte d'argent de l'ordre de 12% contre 10% avec le deuxième modèle (modèle tourné). Par ailleurs avec le premier modèle, la Banque a un risque de perte d'opportunité de gagner de l'argent (intérêts sur les crédits) de l'ordre de 19% contre 24% avec le deuxième modèle.  



### RESUME DES ETAPES DE CONSTRUCTION DU PIPELINE  

✓ Préparation du jeu de données ;  
✓ Première étape du pipeline : Prétraitement (Preprocessing) comprenant transformation des variables catégorielles (Encoding) et Mise à l'échelle (Scaling) ;  
✓ Deuxième étape du pipeline : Réduction de dimensionnalité (Analyse en Composantes Principales) ;  
✓ Troisième étape du pipeline : Modélisation ;  
✓ Quatrième étape : Recherche des meilleurs hyperparamètres (Grid Search).

## CONCLUSION
Dans ce projet j'ai montré, à travers une étude de cas (Modélisation du risque de crédit), qu'un pipeline peut regrouper trois (03) incontournables étapes d'un projet de Data Science à savoir :  
* Pré-traitement et transformation des données ;
* Construction d'un ou de plusieurs modèle(s) de Machine Learning ;
* Recherche des meilleurs hyperparamètres.  

Ainsi, le code de ce projet peut être appliqué à un nouveau jeu de données ce qui fait un gain énorme de temps et d'efficacité.  

Il y a plusieurs autres manières d'utiliser un Pipeline. Je vous conseille de consulter cette [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#examples-using-sklearn-pipeline-pipeline) de Scikit-Learn qui est très riche en exemples pratiques.